<a href="https://colab.research.google.com/github/ranshull/customer-value-retention/blob/main/Product_Customer_Analytics_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df= pd.read_csv("/content/E-commerce Customer Behavior - Sheet1.csv")

In [ ]:
df.head()

,Customer ID,Gender,Age,City,Membership Type,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level
0,101,Female,29,New York,Gold,1120.20,14,4.6,True,25,Satisfied
1,102,Male,34,Los Angeles,Silver,780.50,11,4.1,False,18,Neutral
2,103,Female,43,Chicago,Bronze,510.75,9,3.4,True,42,Unsatisfied
3,104,Male,30,San Francisco,Gold,1480.30,19,4.7,False,12,Satisfied
4,105,Male,27,Miami,Silver,720.40,13,4.0,True,55,Unsatisfied


In [ ]:
df.shape

(350, 11)

In [ ]:
df.columns

Index(['Customer ID', 'Gender', 'Age', 'City', 'Membership Type',
       'Total Spend', 'Items Purchased', 'Average Rating', 'Discount Applied',
       'Days Since Last Purchase', 'Satisfaction Level'],
      dtype='object')

In [ ]:
df.dtypes

,0
Customer ID,int64
Gender,object
Age,int64
City,object
Membership Type,object
Total Spend,float64
Items Purchased,int64
Average Rating,float64
Discount Applied,bool
Days Since Last Purchase,int64


In [ ]:
df.isnull().sum()

,0
Customer ID,0
Gender,0
Age,0
City,0
Membership Type,0
Total Spend,0
Items Purchased,0
Average Rating,0
Discount Applied,0
Days Since Last Purchase,0


In [ ]:
df['Satisfaction Level'].unique()

array(['Satisfied', 'Neutral', 'Unsatisfied', nan], dtype=object)

In [ ]:
df['City'].unique()

array(['New York', 'Los Angeles', 'Chicago', 'San Francisco', 'Miami',
       'Houston'], dtype=object)

In [ ]:
df[df['Satisfaction Level'].isnull()]


,Customer ID,Gender,Age,City,Membership Type,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level
71,172,Female,37,Houston,Bronze,420.8,7,3.1,False,21,NaN
143,244,Female,37,Houston,Bronze,430.8,7,3.4,False,23,NaN


In [ ]:
# Handle missing data
df['Satisfaction Level'] = df['Satisfaction Level'].fillna('Neutral')

In [ ]:
print("\nRating range:")
print(df['Average Rating'].min(), df['Average Rating'].max())

print("\nDays since last purchase range:")
print(df['Days Since Last Purchase'].min(), df['Days Since Last Purchase'].max())


Rating range:
3.0 4.9

Days since last purchase range:
9 63


In [ ]:
# Age group segmentation
df['Age_Group'] = pd.cut(
    df['Age'],
    bins=[17, 25, 35, 45, 100],
    labels=['18-25', '26-35', '36-45', '46+']
)

# Recency status for retention analysis
df['Recency_Status'] = pd.cut(
    df['Days Since Last Purchase'],
    bins=[-1, 30, 60, 365],
    labels=['Active', 'At Risk', 'Churned']
)

# Spend bucket based on distribution
df['Spend_Bucket'] = pd.qcut(
    df['Total Spend'],
    q=3,
    labels=['Low', 'Medium', 'High']
)

# Quick verification
print("Age group distribution:")
print(df['Age_Group'].value_counts())

print("\nRecency status distribution:")
print(df['Recency_Status'].value_counts())

print("\nSpend bucket distribution:")
print(df['Spend_Bucket'].value_counts())


Age group distribution:
Age_Group
26-35    234
36-45    116
18-25      0
46+        0
Name: count, dtype: int64

Recency status distribution:
Recency_Status
Active     226
At Risk    121
Churned      3
Name: count, dtype: int64

Spend bucket distribution:
Spend_Bucket
Low       124
High      117
Medium    109
Name: count, dtype: int64


In [ ]:
# new ranges
df['Age_Group'] = pd.cut(
    df['Age'],
    bins=[25, 30, 35, 40, 45],
    labels=['26-30', '31-35', '36-40', '41-45']
)

print("Updated Age group distribution:")
print(df['Age_Group'].value_counts())


Updated Age group distribution:
Age_Group
26-30    117
31-35    117
36-40     58
41-45     58
Name: count, dtype: int64


SQL Queries

In [ ]:
import duckdb
con = duckdb.connect()

con.register('customers', df)

con.execute("SELECT COUNT(*) FROM customers").fetchall()


[(350,)]

In [ ]:
con.execute("""
SELECT
    COUNT(*) AS total_customers,
    ROUND(AVG("Total Spend"), 2) AS avg_spend,
    ROUND(AVG("Items Purchased"), 2) AS avg_items,
    ROUND(AVG("Days Since Last Purchase"), 2) AS avg_recency
FROM customers
""").fetchdf()



,total_customers,avg_spend,avg_items,avg_recency
0,350,845.38,12.6,26.59


In [ ]:
con.execute("""
SELECT
    "Membership Type" AS membership_type,
    COUNT(*) AS customers,
    ROUND(AVG("Total Spend"), 2) AS avg_spend,
    ROUND(AVG("Items Purchased"), 2) AS avg_items,
    ROUND(AVG("Days Since Last Purchase"), 2) AS avg_recency
FROM customers
GROUP BY "Membership Type"
ORDER BY avg_spend DESC
""").fetchdf()


,membership_type,customers,avg_spend,avg_items,avg_recency
0,Gold,117,1311.14,17.62,17.94
1,Silver,117,748.43,11.66,30.26
2,Bronze,116,473.39,8.49,31.61


In [ ]:
con.execute("""
SELECT
    "Discount Applied" AS discount_applied,
    COUNT(*) AS customers,
    ROUND(AVG("Total Spend"), 2) AS avg_spend,
    ROUND(AVG("Items Purchased"), 2) AS avg_items,
    ROUND(AVG("Average Rating"), 2) AS avg_rating
FROM customers
GROUP BY "Discount Applied"
ORDER BY discount_applied
""").fetchdf()


,discount_applied,customers,avg_spend,avg_items,avg_rating
0,False,175,903.49,13.07,4.06
1,True,175,787.27,12.13,3.98


In [ ]:
con.execute("""
SELECT
    "Satisfaction Level" AS satisfaction_level,
    COUNT(*) AS customers,
    ROUND(AVG("Total Spend"), 2) AS avg_spend,
    ROUND(AVG("Days Since Last Purchase"), 2) AS avg_recency
FROM customers
GROUP BY "Satisfaction Level"
ORDER BY avg_recency
""").fetchdf()


,satisfaction_level,customers,avg_spend,avg_recency
0,Satisfied,125,1280.32,17.70
1,Neutral,109,612.92,19.34
2,Unsatisfied,116,595.14,42.98


In [ ]:
con.execute("""
SELECT
    recency_status,
    COUNT(*) AS customers
FROM customers
GROUP BY recency_status
ORDER BY customers DESC
""").fetchdf()


,Recency_Status,customers
0,Active,226
1,At Risk,121
2,Churned,3


In [ ]:
con.execute("""
SELECT
    "Membership Type" AS membership_type,
    COUNT(*) AS high_risk_customers
FROM customers
WHERE recency_status = 'Churned'
GROUP BY "Membership Type"
ORDER BY high_risk_customers DESC
""").fetchdf()


,membership_type,high_risk_customers
0,Silver,3


In [ ]:
# Save cleaned dataset for Tableau and reporting
df.to_csv("customers_clean.csv", index=False)

print("File saved as customers_clean.csv")


File saved as customers_clean.csv
